In [1]:
import os

In [2]:
current_path=os.getcwd()
current_path

'c:\\Users\\Asus\\Desktop\\all projects & learnings\\llm-rag-fastapi-study'

In [3]:
folder_path="C:\\Users\\Asus\\Documents\\all_doc"

In [4]:
from langchain_community.document_loaders import PyPDFLoader

c:\Users\Asus\Desktop\all projects & learnings\llm-rag-fastapi-study\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
all_doc=[]
for file in os.listdir(folder_path):
    if file.endswith('.pdf'):
        loader=PyPDFLoader(os.path.join(folder_path+"\\"+file))
        doc= loader.load()
        all_doc.extend(doc)

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=10)
chunk=splitter.split_documents(all_doc)
print("splitted into ",len(chunk),"chunks")

splitted into  15 chunks


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [9]:
embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Asus\AppData\Local\Temp\ipykernel_22624\3386098630.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
from langchain_community.vectorstores import FAISS

In [11]:
db=FAISS.from_documents(chunk,embedding)

In [12]:
print("uploaded ",db.index.ntotal,"chunk to vector db")

uploaded  15 chunk to vector db


In [13]:
from langchain_community.llms import Ollama

In [14]:
llm = Ollama(model="llama3.2")

C:\Users\Asus\AppData\Local\Temp\ipykernel_22624\1555928118.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.2")


In [15]:
retrieve= db.as_retriever(search_kwargs={'k':10})

In [16]:
from langchain_core.prompts import ChatPromptTemplate 

In [17]:
template="""Give response only from the below details
{context}
question : {question}
"""
prompt=ChatPromptTemplate.from_template(template)

In [18]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [19]:

rag_chain=(
    {'context': retrieve | (lambda docs : "\n".join( doc.page_content for doc in docs)),'question' : RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [20]:
questions=['list the projects done in meta and teksystems',
           'list the skillset of nivash']

In [21]:
for q in questions:
    print(f"Prompt : {q}")
    answer=rag_chain.invoke(q)
    print(f"Response : {answer}")



Prompt : list the projects done in meta and teksystems
Response : Here are the projects mentioned:

1. LLM Data Pipeline & Annotation Analytics (Meta) - November 2023 to July 2025
2. Recommendation Updates (POC Project) - Refined AI-generated recommendations using iterative prompt strategies, enhancing accuracy and real-time adaptability for users.
3. Dialogue Flow & Intent Optimization – Internal CX Project (POC Project) - Improved chatbot performance by analyzing and creating intents, reducing fallback responses, and enhancing query accuracy.

For Teksystems:
No specific projects are mentioned in the given details.
Prompt : list the skillset of nivash
Response : Here are the skills listed for Nivash:

1. Technical Skills:
   - Python
   - DBMS
   - SQL
   - Data Analysis & Visualization
   - Technical Troubleshooting & Issue Resolution

2. Business & Domain Skills:
   - Banking Operations & Compliance
   - Customer Data Management & Risk Analysis
   - Business Process Improvement

3.